## 🎓 Library

In [1]:
from googletrans import Translator
from tqdm import tqdm
import pandas as pd

DATA = "data/"


## Initial setup

We start with 71570 poems

In [2]:
poems_df = pd.read_parquet(DATA + "de_poems.parquet")

In [3]:
poems_df.head(3)["text"]

0    Gebohrn, und wiederumb, o Mensch, gebohren wer...
1    Es suchte niemand Gott, liegt er gleich unterm...
2    Die ist in Gott, und Gott der ist zugleich in ...
Name: text, dtype: object

In [4]:
new_poems_df = poems_df.copy()

In [58]:
MAX_CHARS = 8500
SEPARATOR = "\t"

async def translate_df(df: pd.DataFrame, src: str = "de", dest: str = "en") -> pd.DataFrame:
    """
    Translates the text of a DataFrame from src to dst language.

    Args:
        df (pd.DataFrame): DataFrame containing the text to be translated.
        src (str): Source language code.
        dest (str): Destination language code.

    Returns:
        pd.DataFrame: DataFrame with translated text.
    """
    translator = Translator()
    translated_rows = []

    buffer = []
    buffer_len = 0
    indices = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Translating", colour="green"):
        text = row["text"]
        text_len = len(text) + len(SEPARATOR)

        # If this text would overflow the buffer, translate current batch first
        if buffer_len + text_len > MAX_CHARS:
            if buffer:  # prevent empty batch
                joined_text = SEPARATOR.join(buffer)
                translated = (await translator.translate(joined_text, src=src, dest=dest)).text
                split_texts = translated.split(SEPARATOR)

                for i, translated_text in zip(indices, split_texts):
                    translated_rows.append({
                        "title": df.at[i, "title"],
                        "text": translated_text,
                        "author": df.at[i, "author"],
                        "creation": df.at[i, "creation"]
                    })

            # Reset buffer
            buffer = []
            indices = []
            buffer_len = 0

        # If single poem is still too long, skip it (optional safety)
        if text_len > MAX_CHARS:
            print(f"Skipping poem {idx} (too long: {text_len} chars)")
            continue

        # Add to buffer
        buffer.append(text)
        indices.append(idx)
        buffer_len += text_len

    # Handle final batch
    if buffer:
        joined_text = SEPARATOR.join(buffer)
        translated = (await translator.translate(joined_text, src=src, dest=dest)).text
        split_texts = translated.split(SEPARATOR)

        for i, translated_text in zip(indices, split_texts):
            translated_rows.append({
                "title": df.at[i, "title"],
                "text": translated_text,
                "author": df.at[i, "author"],
                "creation": df.at[i, "creation"]
            })

    return pd.DataFrame(translated_rows)

# Data merge (Generation)

Merge two datasets into one. This requires the translation of one of the datasets into the language of the other dataset.


## German to English

English poems do not have a creation date, but this will not matter for the generation training.

In [24]:
english_poems = pd.read_parquet(DATA + "en_poems.parquet")

In [66]:
sample = poems_df.head(10000)
de_translated = await translate_df(sample, src="de", dest="en")

Translating:   0%|          | 0/10000 [00:00<?, ?it/s]

Translating:   2%|▏         | 164/10000 [00:01<01:13, 134.10it/s]

Skipping poem 157 (too long: 23508 chars)


Translating:   9%|▉         | 885/10000 [00:12<02:37, 57.79it/s] 

Skipping poem 871 (too long: 14655 chars)


...

Translating:  97%|█████████▋| 9687/10000 [12:18<01:08,  4.56it/s]

Skipping poem 9686 (too long: 13875 chars)


Translating:  97%|█████████▋| 9692/10000 [12:18<01:00,  5.13it/s]

Skipping poem 9691 (too long: 14070 chars)


Translating: 100%|██████████| 10000/10000 [13:26<00:00, 12.40it/s]


In [71]:
sample["text"].iloc[0]

'Gebohrn, und wiederumb, o Mensch, gebohren werden,\nErrettet dich vom Tod und hilfft dir in Beschwerden.'

In [70]:
de_translated["text"][0]

'Drum, and again, o human, being born,\nSalvents you from death and help you in symptoms.'

### Saving

In [ ]:
new_poems_df = pd.concat([english_poems, de_translated], ignore_index=True)

In [76]:
new_poems_df.to_parquet(DATA + "en_de.parquet", index=False)

# Data augmentation (Classification)

Transform the original dataset with new samples generated through different methods. Most libraries like textattack, nlpaug, are not designed for german sentences.

Source: [Text data augmentations: Permutation, antonyms and negation
](https://www.sciencedirect.com/science/article/abs/pii/S0957417421002104)

## Line permutation

Check the file `classification/utils.py` for the function `data_augment`. This function shuffles the lines of poems in the same century.


The model `classification/logistic_regression/tfidf.ipynb` can be trained with the augmented data.

## Translation to english, textattack augment, and back to german

Augmenting with textattack changes proper names and quantities.

In [ ]:
translator = Translator()

In [ ]:
transformed_poems = []
for index, row in new_poems_df.iterrows():
    translated = await translator.translate(row["text"], src="de", dest="en")
    translated_row = {
        "title": row["title"],
        "text": translated.text,
        "author": row["author"],
        "creation": row["creation"]
    }
    transformed_poems.append(translated_row)

translated_df = pd.DataFrame(transformed_poems)

## Germanet synonyms

GermaNet is a semantic network for german, similar to WordNet. It contains information about the meaning of words and their relationships to each other. It can be used to find semantic relations between words, such as synonyms, and antonyms.

Sadly the dataset is not open source and needs to have an authorization to use it.